In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

import operator

from text_processing_udpipe_w2v import get_text_map

from tqdm import tqdm

import statistics 

import json

C:\Users\Asus\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [14]:
dp_300 = pd.read_csv("3000.csv", sep = ",")
dp_300.head()

,Unnamed: 0,texts_3000,same_word_list_3000,percent_3000
0,0,Указом президента России Бориса Ельцина внесен...,"{'отмечать', 'изменение', 'иностранный', 'роль...","71,25"
1,1,В Харькове неизвестные изуродовали лицо депута...,"{'регион', 'серьезный', 'агентство', 'являться...","72,22222222"
2,2,"Руководитель следствия по делу ""Аэрофлота"" ...","{'ставить', 'ход', 'фирма', 'отказываться', 'с...",74
3,3,Постоянный экономический рост позволит Китаю к...,"{'внутренний', 'момент', 'сфера', 'слово', 'мн...","72,60273973"
4,4,В одном из ночных клубов Хмельницкого группа п...,"{'отмечать', 'повод', 'смерть', 'свидетель', '...","70,68965517"


In [2]:
answer_dict = {"222" :['0-' ,'1-' ,'2+' ,'4+'],
"862":['1+','2-', '4+', '6-', '12-'],
"321":['5+' ,'0-' ,'4-' ,'1+'],
"364":['2+', '6-', '3-', '7-' ],
"502":['13-', '3+', '0-', '4+'],
"878":['3-', '0+', '4-', '1-'],
"666":['3-', '1-', '2-' ],
"92":['1+', '2-'],
"615":['1+', '2-'],
"450":['6-', '7-', '11+', '9+' ,'10+' ],
"722":['1+', '6-', '2+', '3+', '6-', '8-'],
"732":['8-', '0-', '3-', '4-', '2+', '1-'],
"611":['1+'],
"251":['5-', '4+', '2+', '1+', '3+']}
list(answer_dict.keys())

['222',
 '862',
 '321',
 '364',
 '502',
 '878',
 '666',
 '92',
 '615',
 '450',
 '722',
 '732',
 '611',
 '251']

In [8]:
text_db = pd.read_csv("text_db.csv", sep = ",", header=None)
text_db.head()

,0,1,2,3,4,5,6,7,8,9
0,0.6503,0.65,0.200000,0.200000,0.007800,0.007800,0.000000,0.093184,0.237405,0.50
1,0.5626,0.72,0.153846,0.307692,0.014890,0.011259,0.000000,0.169718,0.201980,0.40
2,0.4951,0.83,0.166667,0.000000,0.028309,0.012447,0.000000,0.170251,0.190485,0.50
3,0.6321,0.66,0.000000,0.250000,0.013009,0.018568,0.000000,0.223918,0.214968,0.25
4,0.6601,0.62,0.142857,0.214286,0.028585,0.004112,0.004621,0.106511,0.206026,0.50


In [25]:
potential_recommended_text_map = get_text_map("text_8.txt")

In [12]:
def text_features_precalculation (rec_text_map, marked_text_db):
    rec_text_text_similarity_dict = {}
    rec_text_features = []
    rec_text_features.append(rec_text_map['lix'])
    rec_text_features.append(rec_text_map['ttr'])
    rec_text_features.extend(rec_text_map['sent_properties'])
    rec_text_vec = np.array(rec_text_features).reshape(1, -1)
    #print(rec_text_vec)
    text_similarity_dict = {}
    for text_similarity_ind in range(len(marked_text_db)):
        text_vec = marked_text_db.iloc[text_similarity_ind][:-1].values.reshape(1, -1)
        #print(text_vec)
        similarity = cosine_similarity(rec_text_vec,text_vec)
        #print(similarity)
        text_similarity_dict[text_similarity_ind] = similarity[0][0]
    sorted_text_feat_dict = sorted(text_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
    text_feeat_mean_dict = []
    return sorted_text_feat_dict[:min(5, len(sorted_text_feat_dict))]
text_features_precalculation(potential_recommended_text_map, text_db)        

[(2, 0.9610605326200949),
 (10, 0.9488736521244403),
 (7, 0.9464913106739378),
 (0, 0.9451091089484482),
 (13, 0.9414590400696716)]

In [15]:
for text_ind in tqdm(range(len(dp_300))):
    if text_ind not in list(answer_dict.keys()):
        potential_recommended_text_map = get_text_map(dp_300.iloc[text_ind]['texts_3000'], raw_text_input= True)
        STORE_JSON  = text_features_precalculation(potential_recommended_text_map,text_db)
        save_loc = './texts_features_json/text_' + str(text_ind) + ".json"
        with open(save_loc,  'w', encoding = "utf-8") as outfile:  
            json.dump(STORE_JSON, outfile, ensure_ascii=False)

100%|██████████████████████████████████████████████████████████████████████████████| 898/898 [2:39:12<00:00, 10.64s/it]


In [9]:
sent_dp = pd.read_csv("sentence_db.csv", sep = ",", header=None)
sent_dp.head()

,0,1,2,3,4,5,6,7
0,0.000000,0.000000,0.000000,0.000000,0.0,0.213510,0.242857,0.0
1,0.000000,0.000000,0.024805,0.024805,0.0,0.000000,0.277500,0.0
2,0.000000,0.066667,0.000000,0.000000,0.0,0.000000,0.250000,1.0
3,0.076923,0.000000,0.000000,0.000000,0.0,0.138684,0.200000,1.0
4,0.000000,0.071429,0.000000,0.000000,0.0,0.000000,0.238462,1.0


In [26]:
potential_recommended_text_map['sentences_map']

[OrderedDict([('spec_sentence_features',
               OrderedDict([('negation', 0),
                            ('coreference', 0),
                            ('vozvr_verb', 0),
                            ('total_vozvr', 0),
                            ('prich', 0),
                            ('total_prich', 0),
                            ('deepr', 0),
                            ('total_deepr', 0),
                            ('case_complexity', 0),
                            ('total_case', 0),
                            ('mean_depend_length', 0)])),
              ('syntax_prop',
               OrderedDict([('distances_list', []),
                            ('sent_vocab_imp', 2.2160727229152952)])),
              ('sentence_words',
               [OrderedDict([('word', 'В'),
                             ('lemma', 'в'),
                             ('vocabulary_prop',
                              OrderedDict([('tf_idf', 0),
                                           ('nominal

In [30]:
def filter_similar(sorted_similarity):
    filtered_sorted_similarity_list = []
    first_similarity = sorted_similarity[0][1]
    if first_similarity < 0.6:
            return None
    for similar_el in sorted_similarity[:5]:
        similarity_value = similar_el[1]
        if similarity_value - first_similarity > 0.15 or similarity_value < 0.5:
            break
        filtered_sorted_similarity_list.append(similar_el)
    return filtered_sorted_similarity_list

In [31]:
def sent_features_recommendation (rec_text_map, marked_sentences_db):
    sentences_understanding = {}
    sentence_map = rec_text_map['sentences_map']
    for sentence_ind in range(len(sentence_map)):
        rec_sent_feat = []
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['negation'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['coreference'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['vozvr_verb'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['prich'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['deepr'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['case_complexity'])
        rec_sent_feat.append(sentence_map[sentence_ind]['spec_sentence_features']['mean_depend_length'])
        rec_sent_vec = np.array(rec_sent_feat).reshape(1, -1)
        sent_similarity_dict = {}
        for sent_ind in range(len(marked_sentences_db)):
            sent_vec = marked_sentences_db.iloc[sent_ind][:-1].values.reshape(1, -1)
            similarity = cosine_similarity(rec_sent_vec,sent_vec)
            #print(similarity)
            sent_similarity_dict[sent_ind] = similarity[0][0]
        sorted_sent_similarity_dict = sorted(sent_similarity_dict.items(), key=operator.itemgetter(1), reverse = True)
        filtered = filter_similar(sorted_sent_similarity_dict)
        
        """
        sent_similarity_list = []
        for sim_sent in sorted_sent_similarity_dict[:3]:
            print(sim_sent)
            sent_undestanding_mark = list(marked_sentences_db.iloc[sim_sent[0]])[-1]
            sent_similarity_list.append(sent_undestanding_mark)
        """
        sentences_understanding[sentence_ind] = filtered
    return sentences_understanding
sent_features_recommendation(potential_recommended_text_map, sent_dp)        

{0: None,
 1: None,
 2: None,
 3: None,
 4: None,
 5: None,
 6: None,
 7: None,
 8: None,
 9: None,
 10: None,
 11: [(11, 0.6038576879954624)],
 12: None,
 13: [(11, 0.6038576879954624)],
 14: None,
 15: None,
 16: None,
 17: None,
 18: None,
 19: [(11, 0.6038576879954624)],
 20: None,
 21: None,
 22: None}

In [32]:
for text_ind in tqdm(range(len(dp_300))):
    if text_ind not in list(answer_dict.keys()):
        potential_recommended_text_map = get_text_map(dp_300.iloc[text_ind]['texts_3000'], raw_text_input= True)
        STORE_JSON  = sent_features_recommendation(potential_recommended_text_map,sent_dp)
        save_loc = './sentence_features_json/text_' + str(text_ind) + ".json"
        with open(save_loc,  'w', encoding = "utf-8") as outfile:  
            json.dump(STORE_JSON, outfile, ensure_ascii=False)

100%|██████████████████████████████████████████████████████████████████████████████| 898/898 [1:52:14<00:00,  7.50s/it]
